In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

# Running whole Datasource Class for AzureDevops

In [0]:
# imports
from Core.ELTAzureDevops.DatasourceAzureDevopsClass import DatasourceAzureDevopsClass

# init
datasource = DatasourceAzureDevopsClass(dbutils, spark)
bronze_data = datasource.bronzeprocessing(dbutils)

# Calling AzureDevops class to check methods one by one

In [0]:
# imports
from Core.ELTAzureDevops.ServicesAzureDevops.ServiceAzureDevopsBronze import ServiceBronze

# init
bronze = ServiceBronze(dbutils, spark)

In [0]:
pull_requests = bronze.pull_requests(checking_data=True)
display(pull_requests)

In [0]:
branches = bronze.branches(checking_data=True)
display(branches)

In [0]:
branches.select('branch_name').distinct().count()

In [0]:
branches = bronze.branches(checking_data=False)

In [0]:
commits = bronze.commits(checking_data=True)
display(commits)

In [0]:
commits.select('upsert_column').distinct().count()

In [0]:
commits = bronze.commits(checking_data=False)

In [0]:
# repos = bronze.repos(checking_data=True)
# display(repos)

In [0]:
# repos = bronze.repos(checking_data=False)

In [0]:
builds = bronze.builds(checking_data=True, test_one_build=False)
display(builds)

In [0]:
builds['id'].unique().size

In [0]:
len(builds)

In [0]:
builds = bronze.builds(checking_data=False, test_one_build=False)

In [0]:
# projects = bronze.projects(checking_data=True)
# display(projects)

In [0]:
# projects = bronze.projects(checking_data=False)

In [0]:
# pipeline_runs = bronze.pipeline_runs(checking_data=True)
# display(pipeline_runs)

In [0]:
# approvals = bronze.approvals(checking_data=True)
# display(approvals)

In [0]:
# definitions = bronze.definitions(checking_data=True)
# display(definitions)

In [0]:
# pipelines = bronze.pipelines(checking_data=True)
# display(pipelines)

In [0]:
# source_providers = bronze.source_providers()
# display(source_providers)

# Table creation

In [0]:
%sql
create table if not exists engineering_metrics.bronze.azure_devops_projects
          (
          id STRING,
          name STRING,
          url STRING,
          state STRING,
          revision STRING,
          visibility STRING,
          lastUpdateTime STRING, -- last_udate_time
          description string
          )
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_projects';

create table if not exists engineering_metrics.silver.azure_devops_projects
          (
          id STRING,
          name STRING,
          url STRING,
          state STRING,
          revision STRING,
          visibility STRING,
          last_udate_time STRING,
          description string
          )
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_projects'

In [0]:
%sql
create table if not exists engineering_metrics.bronze.azure_devops_pipeline_runs
          (
          _links STRING,
          pipeline STRING,
          state STRING,
          result string,
          createdDate STRING,
          finishedDate STRING,
          url STRING,
          id STRING,
          name STRING,
          variables STRING,
          issue_key STRING,
          env string,
          rank string,
          first_is_succeeded string,
          extracted_name_for_issue_key string
          )
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_pipeline_runs';

create table if not exists engineering_metrics.silver.azure_devops_pipeline_runs
  (
        environment string,
        pipeline string,
        state string,
        url string,
        id string,
        pipeline_id string,
        result string,
        created_date timestamp,
        finished_date timestamp,
        run_time_minutes string,
        issue_key string,
        extracted_name_for_issue_key string,
        rank_from_finished_date_asc string,
        first_success boolean
            )
  using delta
  LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_pipeline_runs';

create table if not exists engineering_metrics.silver.azure_devops_pipeline_runs_agg
          (
        issue_key string,
        pipeline_id string,
        successful_attempt string,
        avg_run_time_for_succeeded_min string,
        first_succeeded_build_time_min string,
        minutes_with_first_success string
          )
  using delta
  LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_pipeline_runs_agg';


create table if not exists engineering_metrics.silver.azure_devops_pipeline_runs_agg_minutes_env
          (
environment string,
 summed_run_time_minutes int
          )
  using delta
  LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_pipeline_runs_agg_minutes_env';


create table if not exists engineering_metrics.silver.azure_devops_pipeline_runs_agg_minutes_issue_key
          (
issue_key string,
summed_run_time_minutes int
          )
  using delta
  LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_pipeline_runs_agg_minutes_issue_key';


In [0]:
%sql
create table if not exists engineering_metrics.bronze.azure_devops_approvals
        (
        id STRING,
        steps STRING,
        status STRING,
        createdOn STRING,
        lastModifiedOn STRING,
        executionOrder STRING,
        minRequiredApprovers STRING,
        blockedApprovers STRING,
        _links STRING,
        pipeline STRING,
        instructions string
        )
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_approvals';

create table if not exists engineering_metrics.silver.azure_devops_approvals
          (
      ID string,
      STATUS string,
      pipeline_id string,
      pipeline_name  string,
      pipeline_owner_id string,
      created_on timestamp,
      min_required_approvers string
          )
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_approvals'


In [0]:
%sql
create table if not exists engineering_metrics.bronze.azure_devops_repos
          (
          id STRING,
          name STRING,
          url STRING,
          project STRING,
          defaultBranch STRING,
          size STRING,
          remoteUrl STRING,
          sshUrl STRING,
          webUrl STRING,
          isDisabled STRING,
          isInMaintenance string,
          isFork string,
          proj_id STRING,
          project_name string, 
          upsert_column string
          )
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_repos';

create table if not exists engineering_metrics.silver.azure_devops_repos
(
repo_id string,
    URL string,
    NAME string,
    SIZE string,
is_disabled boolean,
    default_branch string,
 is_in_maintenance boolean,
    PROJECT string          )
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_repos';


In [0]:
%sql
create table if not exists engineering_metrics.bronze.azure_devops_definitions
          (
          _links STRING,
          quality STRING,
          authoredBy STRING,
          drafts STRING,
          queue STRING,
          id STRING,
          name STRING,
          url STRING,
          uri STRING,
          path STRING,
          type STRING,
          queueStatus STRING, --queue_status
          revision STRING,
          createdDate STRING,
          project STRING,
          draftOf STRING,
          projectId STRING, --project_id
          projectName STRING --project_name
          )
using delta
location xyz

LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_definitions'



%sql

INSERT overwrite INTO engineering_metrics.silver.azure_devops_definitions
SELECT
    ID string,
    URL string,
    NAME string,
    PATH string,
    TYPE string,
    REVISION string,
    CREATEDDATE timestamp,
    queue_status string,
    project_id string,
    project_name string,
FROM engineering_metrics.bronze.azure_devops_definitions;



In [0]:
%sql
create table if not exists engineering_metrics.bronze.azure_devops_pipelines
          (
          _links STRING,
          url STRING,
          id STRING,
          revision STRING,
          name STRING,
          folder STRING,
          project_id string
          )
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_pipelines'

%sql

INSERT overwrite INTO engineering_metrics.SILVER.azure_devops_PIPELINES
SELECT
    ID string,
    URL string,
    NAME string,
    FOLDER string,
    REVISION string,
        project_id string,
FROM engineering_metrics.bronze.azure_devops_PIPELINES;

In [0]:
%sql
create table if not exists engineering_metrics.bronze.azure_devops_source_providers
          (
          name STRING,
          supportedTriggers STRING,
          supportedCapabilities STRING,
          upsert_key string
          )
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_source_providers'


-- |-- name: string (nullable = true)
--  |-- supportedTriggers: string (nullable = true)
--  |-- supportedCapabilities: string (nullable = true)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS engineering_metrics.bronze.azure_devops_builds
(
  _links STRING,
  properties STRING,
  tags STRING,
  validationResults STRING,
  plans STRING,
  triggerInfo STRING,
  id STRING,
  buildNumber STRING,
  status STRING,
  result STRING,
  queueTime STRING,
  startTime STRING,
  finishTime STRING,
  url STRING,
  definition STRING,
  project STRING,
  uri STRING,
  sourceBranch STRING,
  sourceVersion STRING,
  queue STRING,
  priority STRING,
  reason STRING,
  requestedFor STRING,
  requestedBy STRING,
  lastChangedDate STRING,
  lastChangedBy STRING,
  parameters STRING,
  orchestrationPlan STRING,
  logs STRING,
  repository STRING,
  retainedByRelease STRING,
  triggeredByBuild STRING,
  appendCommitMessageToRunName STRING,
  project_name STRING,
  repo_id STRING,
  templateParameters STRING,
  buildNumberRevision STRING
)
USING delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_builds';

create table if not exists engineering_metrics.silver.azure_devops_builds (
    _links string,
    properties string,
    tags string,
    validation_results string,
    plans string,
    trigger_info string,
    id string,
    build_number string,
    status string,
    result string,
    queue_time string,
    start_time string,
    finish_time string,
    url string,
    definition string,
    project string,
    uri string,
    source_branch string,
    source_version string,
    queue string,
    priority string,
    reason string,
    requested_for string,
    requested_by string,
    last_changed_date string,
    last_changed_by string,
    parameters string,
    orchestration_plan string,
    logs string,
    repository string,
    retained_by_release string,
    triggered_by_build string,
    append_commit_message_to_run_name string,
    project_name string,
    repo_id string,
    template_parameters string,
    build_number_revision string
)
USING delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_builds';

In [0]:
%sql
select * from engineering_metrics.bronze.azure_devops_pull_requests

In [0]:
%sql
CREATE TABLE IF NOT EXISTS engineering_metrics.bronze.azure_devops_pull_requests
(
  repository STRING,
  pullRequestId STRING, --pull_request_id
  codeReviewId STRING, --code_review_id
  status STRING,
  createdBy STRING, --created_by
  creationDate STRING, --creation_date
  closedDate STRING, --closed_date
  title STRING,
  description STRING,
  sourceRefName STRING, --source_ref_name
  targetRefName STRING, --target_ref_name
  mergeStatus STRING, --merge_status
  isDraft STRING, --is_draft
  mergeId STRING,
  lastMergeSourceCommit STRING, -- last_merge_source_commit
  lastMergeTargetCommit STRING, --last_merge_target_commit
  lastMergeCommit STRING, --last_merge_commit
  reviewers STRING,
  url STRING,
  completionOptions STRING, --completion_options
  supportsIterations STRING, --supports_iterations
  completionQueueTime STRING, --completion_queue_time
  project_name STRING,
  repo_id STRING,
  autoCompleteSetBy STRING, --auto_completee_set_by
  upsert_column string
)
USING delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_pull_requests';

create table if not exists engineering_metrics.silver.azure_devops_pull_requests
(
    repository string,
    pull_request_id string,
    code_review_id string,
    status string,
    created_by string,
    creation_date string,
    closed_date string,
    title string,
    description string,
    source_ref_name string,
    target_ref_name string,
    merge_status string,
    is_draft string,
    merge_id string,
    last_merge_source_commit string,
    last_merge_target_commit string,
    last_merge_commit string,
    reviewers string,
    url string,
    completion_options string,
    supports_iterations string,
    completion_queue_time string,
    project_name string,
    repo_id string,
    auto_complete_set_by string
)
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_pull_requests';

create table if not exists engineering_metrics.silver.azure_devops_aged_pull_requests
(
    one_day_tickets bigint,
    three_day_tickets bigint,
    seven_day_tickets bigint,
    older_tickets bigint
)
using delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_aged_pull_requests';






-- CREATE OR REPLACE TABLE engineering_metrics.silver.azure_devops_PULL_REQUESTS_SNOWFLAKE
-- AS
-- WITH base_data AS (
--     SELECT
--         pull_request_id,
--         merge_id,
--         title,
--         description,
--         status,
--         is_draft,
--         CAST(creation_date AS DATETIME) AS creation_date,
--         CAST(closed_date AS DATETIME) AS closed_date,
--         DATEDIFF(MINUTE, creation_date, closed_date) AS age,
--         merge_status,
--         created_by,
--         repository,
--         timestamp
-- --        reviewers
--     FROM engineering_metrics.silver.azure_devops_pull_requests
-- )

-- SELECT
--     pull_request_id,
--     merge_id,
--     title,
--     description,
--     status,
--     is_draft,
--     creation_date,
--     closed_date,
--     age,
--     merge_status,
--     try_parse_json(created_by):displayName AS created_by,
--     try_parse_json(repository):name AS repository,
--     try_parse_json(repository):id AS repository_id
-- FROM base_data;

-- %sql
-- INSERT overwrite INTO engineering_metrics.silver.azure_devops_AGED_PULL_REQUESTS
-- SELECT
--     SUM(CASE WHEN DATEDIFF(DAY, TO_TIMESTAMP_TZ(REGEXP_REPLACE(CREATIONDATE, '(\\d{2})(\\d{2})$', '\\1:\\2')), CURRENT_TIMESTAMP()) < 1 THEN 1 ELSE 0 END) AS one_day_tickets,
--     SUM(CASE WHEN DATEDIFF(DAY, TO_TIMESTAMP_TZ(REGEXP_REPLACE(CREATIONDATE, '(\\D{2})(\\D{2})$', '\\1:\\2')), CURRENT_TIMESTAMP()) BETWEEN 1 AND 3 THEN 1 ELSE 0 END) AS three_day_tickets,
--     SUM(CASE WHEN DATEDIFF(DAY, TO_TIMESTAMP_TZ(REGEXP_REPLACE(CREATIONDATE, '(\\D{2})(\\D{2})$', '\\1:\\2')), CURRENT_TIMESTAMP()) BETWEEN 4 AND 7 THEN 1 ELSE 0 END) AS seven_day_tickets,
--     SUM(CASE WHEN DATEDIFF(DAY, TO_TIMESTAMP_TZ(REGEXP_REPLACE(CREATIONDATE, '(\\D{2})(\\D{2})$', '\\1:\\2')), CURRENT_TIMESTAMP()) > 7 THEN 1 ELSE 0 END) AS older_tickets
-- FROM engineering_metrics.silver.azure_devops_pull_requests
-- WHERE STATUS = 'active';

In [0]:
%sql
CREATE TABLE IF NOT EXISTS engineering_metrics.bronze.azure_devops_branches
(
  repository_id STRING,
  project_name STRING,
  branch_name STRING,
  branch_full_name STRING,
  upsert_column string
)
USING delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_branches';


CREATE TABLE IF NOT EXISTS engineering_metrics.silver.azure_devops_branches
(
  project_name string,
    repository_id string,
    branch_name string
    )
USING delta
LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_branches';

In [0]:
%sql
create table if not exists engineering_metrics.bronze.azure_devops_commits
          (
            commitId STRING,
            author STRING,
            committer STRING,
            comment STRING,
            changeCounts STRING,
            url STRING,
            remoteUrl STRING,
            repository_id STRING,
            branch_name STRING,
            project_name STRING,
            commentTruncated STRING,
            upsert_column string
          )
 using delta
 LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/bronze/azure_devops_commits';


  create table if not exists engineering_metrics.silver.azure_devops_commits
          (
          repository_id string,
          branch_name string,
          commit_id string,
          author_email string,
          datetime string,
          comment string
          )
 using delta
 LOCATION 'abfss://prod@deltalakeeunprod.dfs.core.windows.net/deltalake/engineering_metrics/silver/azure_devops_commits';
